#Email filtering code
#### (please upload the sample dataset file from github to run this code)

In [15]:
import csv
import re

def filter_emails(csv_file):
    filtered_emails = []
    # please upload the csv file from github to colab to make sure the code runs
    with open(csv_file, 'r') as csvfile, open('filtered_emails.txt', 'w') as outfile:
        reader = csv.reader(csvfile)
        writer = csv.writer(outfile)

        next(reader)  # Skip header row

        for row in reader:
            date, sender, subject, message = row

            interview_keywords = ["interview", "invitation", "screening", "phone interview", "onsite"]
            is_interview = any(keyword in (subject.lower() + message.lower()) for keyword in interview_keywords)

            finance_keywords = ["finance", "accounting", "analyst"]
            is_finance = any(keyword in (subject.lower() + message.lower()) for keyword in finance_keywords)

            company_regex = r"(?:at|from) (.*? Inc\.|Corp\.|LLC)"
            company_match = re.search(company_regex, message, re.IGNORECASE)
            company = company_match.group(1) if company_match else "*To be determined*"

            if is_interview or is_finance:
                role = "*To be determined*"
                writer.writerow([date, sender, subject, message, company, role])
                filtered_emails.append(f"Date: {date}\nSender: {sender}\nSubject: {subject}\nMessage: {message}\nCompany: {company}\nRole: {role}\n")

    print("Filtered emails saved to 'filtered_emails.txt'")
    return '\n'.join(filtered_emails)

output_variable = filter_emails('sample_mail_data.csv')
#You can find the filtered_emails.txt file in the colab files

Filtered emails saved to 'filtered_emails.txt'


##Necessary libraries for AI chat bot made with the help of vertex AI

In [16]:
from google.auth.transport.requests import Request
from google.oauth2.service_account import Credentials

In [19]:
key_path='/content/mykey.json' #upload the key from github to colab and run this.

In [5]:
credentials = Credentials.from_service_account_file(
    key_path,
scopes=['https://www.googleapis.com/auth/cloud-platform'])
if credentials.expired:
  credentials.refresh(Request())

In [6]:
PROJECT_ID='metal-dimension-423109-s9'
REGION='us-central1'

##AI chat bot code

In [18]:
import base64
import vertexai
# initialize vertex
vertexai.init(project = PROJECT_ID, location = REGION, credentials = credentials)
from vertexai.generative_models import GenerativeModel, Part, FinishReason
import vertexai.preview.generative_models as generative_models

def generate():
  vertexai.init(project="metal-dimension-423109-s9", location="us-central1")
  model = GenerativeModel("gemini-1.0-pro-002")
  input_prompt=input("Enter a prompt: ")
  if(input_prompt=='stop'):
    print("If you have any other queries, please restart the bot, I am always here to help!")
    return
  responses = model.generate_content(
      [f"""{output_variable}

input: did i get any mail from microsoft?
output: Yes, you received two emails from Microsoft on 2024-12-05 at 17:24:10 and 17:20:00.
The first email informs you that you have successfully passed the initial screening phase and have been selected to move forward in the selection process for a role at Microsoft. The email highlights your strong qualifications and potential to make a significant impact at the company.
The second email congratulates you on being shortlisted for a role at Microsoft. It emphasizes your impressive qualifications and experience, and expresses the company\'s belief that you would be a valuable addition to their team.
Both emails mention that the next step will involve scheduling interviews to explore your skills, experiences, and fit for the role in greater detail. You can expect to receive an email from Microsoft\'s recruiting team shortly to coordinate interview dates and times.

input: did i get any mail from apple?
output: I'm sorry, but there are no emails in this list from Apple as of now. Please check later.

input: am i selected for any comapny's interview?
output: Yes, you have been shortlisted for interviews at the following companies:
      DEEP TECH for the position of [Job Title] on [Date] at [Time].
      NV NATIONAL for the position of tester on [Date] at [Time].
      AKUTECH for the position of analyst on [Date] at [Time].
      MK GLOBAL for the position of SDE on [Date] at [Time].

input: {input_prompt}
output:
"""],
      generation_config=generation_config,
      safety_settings=safety_settings,
  )

  for candidate in responses.candidates:
        print(candidate.content.parts[0].text)
  generate()

generation_config = {
    "max_output_tokens": 2048,
    "temperature": 1,
    "top_p": 1,
}

safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}

generate()
#the given context above is generated by the mail classification which filters the interview related mails separately.
# the chat bot made from vertex ai answers the questions based on the context given. you can ask a question like "did i recieve any mail from apple?" etc and the bot replies.
# to stop the bot, please input "stop"

Enter a prompt: hello
Hello! How can I help you today?
Enter a prompt: did i get any mail from apple?
I'm sorry, but there are no emails in this list from Apple as of now. Please check later.
Enter a prompt: give me a summary of how many interviws i got selected?
## Interview Summary

Based on the emails you provided, you have been selected for interviews at the following companies and roles:

**Company** | **Role** | **Date** | **Time**
---|---|---|---|
DEEP TECH | [Job Title] | [Date] | [Time]
NV NATIONAL | tester | [Date] | [Time]
AKUTECH | analyst | [Date] | [Time]
MK GLOBAL | SDE | [Date] | [Time]

In addition, you have been shortlisted for interviews at Microsoft for two different roles. The emails do not specify the date and time of the interviews, but you should expect to receive an email from Microsoft's recruiting team shortly to coordinate these details.

Here is a summary of the total number of interview invitations you have received:

* Confirmed interviews: 4
* Shortliste